# Datagedrochten
Deze handelingen ga je uitvoeren in een Jupyter Notebook. Niemand is nog bekend met dit datatype, dus zorg ervoor dat je goed documenteert wat je bevindingen zijn en hoe je tot je code bent gekomen.
Zorg er ook voor dat je de relevante tussenstappen en bevindingen allemaal in je Jupyter Notebook zet. Het is prima om bepaalde aannames of stukken kennis te gebruiken over de database, maar zorg ervoor dat je in je Notebook uitlegt hoe je aan die kennis komt.
Verwerk de data op een logische manier in een MySQL-database.

Allereerst bekijken we de inhoud van het bestand; dit is voornamelijk leesbare tekst en vooralsnog gecodeerd met utf-8.
Het lijkt erop dat er steeds een aantal scheidingstekens (=) gebruikt wordt tussen belangrijke onderdelen van de database.

Deze onderdelen ga ik eerst in verschillende variabelen plaatsen..

In [1]:
import re
import mysql.connector
total_data = []
content = []
with open('data.brm','rt') as file:
    for line in file:
        content.append(line)
        if x:= re.search(r"\={2,}",line):
            total_data.append(content)
            content=[]
                    
    total_data.append(content)


we hebben nu een lijst waarin de verschillende onderdelen inzitten:
als we deze lijst bekijken onderscheid ik de volgende onderdelen.
- introductie met uitleg van de database (=db_readme)
- een flink aantal items welke verschillende attributen bevat (=db_items)
- een verzameling keys behorende bij de items (=db_keys)
- een verzameling values behorende bij de items (=db_values)

Deze onderdelen ga ik in een eigen variabel zetten met bovenstaande namen:


In [2]:
db_readme = total_data[1]
db_items = total_data[2]
db_keys = total_data[3]
db_values = total_data[4]


Dan nu tijd om de verschillende onderdelen klaar te maken voor de sql database.
Ik start met db_keys:

In [3]:
db_keys

['KEYS\n',
 'A => casting_time\n',
 'B => desc\n',
 'C => duration\n',
 'D => level\n',
 'E => name\n',
 'F => range\n',
 'G => school\n',
 'H => spell\n',
 '\n',
 '=======================\n']

Dat ziet eruit als een dictionary; met duidelijke key en value waarden. Hieronder schoon ik de lijst op mbv regex en vul ik een python dict met de opgeschoonde waarden:

De keywoorden 'desc' en 'range' zijn sleutelwoorden, en mogen niet gebruikt worden in een sql-database. 
In de complete database vinden we geen verwijzing naar E(name). Die halen we dan ook uit de 
Deze passen we dus ook handmatig aan.

In [4]:
regex_key = (r"([A-Z])\s=>\s([A-Za-z_]+)")
dict_keys={}
for item in db_keys:
    if x:= re.match(regex_key,item):
        dict_keys[x[1]]=x[2]

dict_keys['B'] = "descend"
dict_keys['F'] = "item_range"
dict_keys.pop("E")
dict_keys

{'A': 'casting_time',
 'B': 'descend',
 'C': 'duration',
 'D': 'level',
 'F': 'item_range',
 'G': 'school',
 'H': 'spell'}

dan de db_values:

In [5]:
db_values

['VALUES\n',
 '000 => 0\n',
 '001 => 1\n',
 '002 => 1 action\n',
 '003 => 1 bonus action\n',
 '004 => 1 hour\n',
 '005 => 1 mile\n',
 '006 => 1 minute\n',
 '007 => 1 reaction\n',
 '008 => 1 round\n',
 '009 => 10 days\n',
 '010 => 10 feet\n',
 '011 => 10 minutes\n',
 '012 => 100 feet\n',
 '013 => 12 hours\n',
 '014 => 120 feet\n',
 '015 => 150 feet\n',
 '016 => 2\n',
 '017 => 24 hours\n',
 '018 => 3\n',
 '019 => 30 days\n',
 '020 => 30 feet\n',
 '021 => 300 feet\n',
 '022 => 4\n',
 '023 => 5\n',
 '024 => 5 feet\n',
 '025 => 500 feet\n',
 '026 => 500 miles\n',
 '027 => 6\n',
 '028 => 60 feet\n',
 '029 => 7\n',
 '030 => 7 days\n',
 '031 => 8\n',
 '032 => 8 hours\n',
 '033 => 9\n',
 '034 => 90 feet\n',
 "035 => A 10-foot-radius immobile dome of force springs into existence around and above you and remains stationary for the duration. The spell ends if you leave its area.\\nNine creatures of Medium size or smaller can fit inside the dome with you. The spell fails if its area includes a larg

Dit ziet er, ondanks de hoeveelheid tekst, ook uit als een duidelijke key/value verhouding. Op bijna dezelfde manier kunnen we ook deze dictionary gaan vullen..

In [6]:
regex_key = (r"(\d{3})\s+=>\s+(.+)")
dict_values={}
for item in db_values:
    if x:= re.match(regex_key,item):
        key= x[1].zfill(3)
        value = x[2].strip()
        dict_values[x[1]] = value
dict_values

{'000': '0',
 '001': '1',
 '002': '1 action',
 '003': '1 bonus action',
 '004': '1 hour',
 '005': '1 mile',
 '006': '1 minute',
 '007': '1 reaction',
 '008': '1 round',
 '009': '10 days',
 '010': '10 feet',
 '011': '10 minutes',
 '012': '100 feet',
 '013': '12 hours',
 '014': '120 feet',
 '015': '150 feet',
 '016': '2',
 '017': '24 hours',
 '018': '3',
 '019': '30 days',
 '020': '30 feet',
 '021': '300 feet',
 '022': '4',
 '023': '5',
 '024': '5 feet',
 '025': '500 feet',
 '026': '500 miles',
 '027': '6',
 '028': '60 feet',
 '029': '7',
 '030': '7 days',
 '031': '8',
 '032': '8 hours',
 '033': '9',
 '034': '90 feet',
 '035': "A 10-foot-radius immobile dome of force springs into existence around and above you and remains stationary for the duration. The spell ends if you leave its area.\\nNine creatures of Medium size or smaller can fit inside the dome with you. The spell fails if its area includes a larger creature or more than nine creatures. Creatures and objects within the dome when

Om straks de SQL-database te vullen moeten we definieren hoe groot de VARCHAR moet zijn..aangezien in deze dict_values de grootste values zitten ga ik met de volgende formule bepalen hoe groot de langste key is :

In [7]:
max_dict_value = max((len(value)) for key,value in dict_values.items())
print(max_dict_value)

3790


Met db_readme doen we niets.. Blijft over db_items

In [8]:
db_items
len(db_items)

322

Ik concludeer hieruit dat er 322 entry's zijn.
Elke entry bevat 7 (A tm H, behalve E) eigenschappen met een bijhorende waarde. Deze eigenschappen en waarden vinden we terug in de key en value dictionaries.
Laten we naar de eerste entry kijken..:

In [9]:
print("Eerste entry : ",db_items[1])
print("H = ",dict_keys['H'])
print("84 = ",dict_values['084'])

Eerste entry :  (H) => [84] || (B) => [65]  (F) => [34]  (C) => [309]   (D) => [16](A) => [2](G) => [224]

H =  spell
84 =  Acid Arrow


Eigenschap H komt overeen met "Spell", bijbehorende waarde 84 komt overeen met 'Acid Arrow'
Al deze entries:
- sorteer ik op alfabet, 
- de bijbehorende waarde uit de values list zet ik in een list waardoor het klaar is om in een sql-database gezet te worden.

In [10]:
regex_key = (r"\(([A-Z])\)\s=>\s\[(\d+)\]")
entries_list = []
temp_list = []
for item in db_items[1:-2]: #De eerste en laatste 2 rijen zijn geen data
    result = re.findall(regex_key,item) # krijg 2 groepen per gevonden resultaat
    result = sorted(result) #alles gesorteerd 
    for items in result:
        value = items[1].zfill(3)
        temp_list.append(dict_values[value])  #Vul de tijdeljke lijst met overeenkomstige waarde uit dict_values
    entries_list.append(temp_list)
    temp_list = []
entries_list

[['1 action',
  'A shimmering green arrow streaks toward a target within range and bursts in a spray of acid. Make a ranged spell attack against the target. On a hit, the target takes 4d4 acid damage immediately and 2d4 acid damage at the end of its next turn. On a miss, the arrow splashes the target with acid for half as much of the initial damage and no damage at the end of its next turn.',
  'Instantaneous',
  '2',
  '90 feet',
  'Evocation',
  'Acid Arrow'],
 ['1 action',
  "You hurl a bubble of acid. Choose one creature within range, or choose two creatures within range that are within 5 feet of each other. A target must succeed on a dexterity saving throw or take 1d6 acid damage.\\nThis spell's damage increases by 1d6 when you reach 5th level (2d6), 11th level (3d6), and 17th level (4d6).",
  'Instantaneous',
  '0',
  '60 feet',
  'Conjuration',
  'Acid Splash'],
 ['1 action',
  "Your spell bolsters your allies with toughness and resolve. Choose up to three creatures within range

Dan hebben we nu 3 schone lijsten met data...
Nu kunnen we een database en tabel gaan maken met de data.

Natuurlijk weten we wat de kolommen zijn; toch wil ik proberen deze te automatiseren.

In [12]:

db_name = "mysteries"

mydb = mysql.connector.connect(host="localhost",user="bit_academy",password="bit_academy")
mycursor = mydb.cursor()
mycursor.execute(f"DROP DATABASE IF EXISTS {db_name}")
mycursor.execute(f"CREATE DATABASE {db_name}")
mycursor.execute(f"USE {db_name}")

#create new table and fill it with rows (header is set from list)
sql_table_drop = "DROP TABLE IF EXISTS mystery_items"
sql_table_create = """CREATE TABLE mystery_items ("""


for key in dict_keys.keys():
    column_text = (f"{dict_keys[key]} TEXT({max_dict_value}) NOT NULL,")
    sql_table_create += column_text
sql_table_create = sql_table_create[:-1] + ")"

print(sql_table_create)

mycursor.execute(sql_table_drop)
mycursor.execute(sql_table_create)


sql_add_entry = "INSERT INTO mystery_items VALUES ("

for column in range(len(dict_keys)):
    sql_add_entry += "%s,"

sql_add_entry = sql_add_entry[:-1] + ")" 
print(sql_add_entry)
mycursor.executemany(sql_add_entry,entries_list)
mydb.commit()
if mycursor.rowcount == len(entries_list):
    print("Klaar!")

CREATE TABLE mystery_items (casting_time TEXT(3790) NOT NULL,descend TEXT(3790) NOT NULL,duration TEXT(3790) NOT NULL,level TEXT(3790) NOT NULL,item_range TEXT(3790) NOT NULL,school TEXT(3790) NOT NULL,spell TEXT(3790) NOT NULL)
INSERT INTO mystery_items VALUES (%s,%s,%s,%s,%s,%s,%s)
Klaar!
